In [ ]:
import input.greatbarrierreef as gbr

In [ ]:
env = gbr.make_env()

In [ ]:
iter_test = env.iter_test()

In [ ]:
from PIL import Image

for (img, df) in iter_test:
    display(Image.fromarray(img))
    df['annotations'] = '0.5 0 0 100 100'
    env.predict(df)

In [ ]:
import argparse
import os
import sys
from pathlib import Path

import cv2
import torch
import torch.backends.cudnn as cudnn

FILE = Path(os.getcwd()).resolve()
ROOT = FILE.parents[0]  # YOLOv5 root directory
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))  # add ROOT to PATH
ROOT = Path(os.path.relpath(ROOT, Path.cwd()))  # relative

from models.common import DetectMultiBackend
from utils.datasets import IMG_FORMATS, VID_FORMATS, LoadImages, LoadStreams
from utils.general import (LOGGER, check_file, check_img_size, check_imshow, check_requirements, colorstr,
                           increment_path, non_max_suppression, print_args, scale_coords, strip_optimizer, xyxy2xywh)
from utils.plots import Annotator, colors, save_one_box
from utils.torch_utils import select_device, time_sync

In [ ]:
torch.cuda.is_available()

In [ ]:
weights='runs/train/yolov5x_fold08/weights/best.pt'  # model.pt path(s)
source='convertor/fold0/images/val2017'  # file/dir/URL/glob, 0 for webcam
imgsz=(720, 1280)  # inference size (height, width)
conf_thres=0.25  # confidence threshold
iou_thres=0.45  # NMS IOU threshold
max_det=1000  # maximum detections per image
device='cuda:0'  # cuda device, i.e. 0 or 0,1,2,3 or cpu
view_img=False  # show results
save_txt=False  # save results to *.txt
save_conf=False  # save confidences in --save-txt labels
save_crop=False  # save cropped prediction boxes
nosave=False  # do not save images/videos
classes=None  # filter by class: --class 0, or --class 0 2 3
agnostic_nms=False  # class-agnostic NMS
augment=False  # augmented inference
visualize=False  # visualize features
update=False  # update all models
project='runs/detect'  # save results to project/name
name='exp'  # save results to project/name
exist_ok=False  # existing project/name ok, do not increment
line_thickness=3  # bounding box thickness (pixels)
hide_labels=False  # hide labels
hide_conf=False  # hide confidences
half=False  # use FP16 half-precision inference
dnn=False  # use OpenCV DNN for ONNX inference


# Load model
# device = select_device(device)
print(device)
model = DetectMultiBackend(weights, device=device, dnn=dnn)
stride, names, pt, jit, onnx, engine = model.stride, model.names, model.pt, model.jit, model.onnx, model.engine
imgsz = check_img_size(imgsz, s=stride)  # check image size

# Half
# half &= (pt or jit or engine) and device.type != 'cpu'  # half precision only supported by PyTorch on CUDA
# if pt or jit:
    # model.model.half() if half else model.model.float()

# Dataloader
# if webcam:
    # view_img = check_imshow()
    # cudnn.benchmark = True  # set True to speed up constant image size inference
    # dataset = LoadStreams(source, img_size=imgsz, stride=stride, auto=pt)
    # bs = len(dataset)  # batch_size
# else:
    # dataset = LoadImages(source, img_size=imgsz, stride=stride, auto=pt)
    # bs = 1  # batch_size
# vid_path, vid_writer = [None] * bs, [None] * bs

# Run inference
with torch.no_grad():
    model.warmup(imgsz=(1, 3, *imgsz), half=half)  # warmup
    dt, seen = [0.0, 0.0, 0.0], 0
# for path, im, im0s, vid_cap, s in dataset:
    # t1 = time_sync()

In [ ]:
import glob
import cv2
from PIL import Image

In [ ]:
import numpy as np
import pandas as pd
df = pd.read_csv('input/train.csv')
df = df[['image_id','annotations']]

In [ ]:
import ast

impath = glob.glob("convertor/fold0/images/val2017/*.jpg")
# path = impath[103]
for path in impath[:20]:
    im = cv2.imread(path)
    display(Image.fromarray(cv2.cvtColor(im,cv2.COLOR_BGR2RGB)))
    im = cv2.resize(im, dsize=(1280,736))
    img0 = im.copy()

    im = im.transpose((2, 0, 1))[::-1] # convert to CHW, RGB

    im = torch.from_numpy(im.copy()).to(device)
    im = im.half() if half else im.float()  # uint8 to fp16/32
    im /= 255  # 0 - 255 to 0.0 - 1.0
    if len(im.shape) == 3:
        im = im[None]  # expand for batch dim
    # print(im.shape)

    # Inference
    with torch.no_grad():
        pred = model(im)
        pred = non_max_suppression(pred, conf_thres, iou_thres, classes, agnostic_nms, max_det=max_det) # NMS
        # pred = pred[0].cpu().numpy()
        for det in pred:
            det[:, :4] = scale_coords(im.shape[2:], det[:, :4], img0.shape).round()

            for bbox in det:
                bbox = bbox.cpu().numpy()
                x,y,x2,y2 = bbox[:4].astype(np.uint16)
                cv2.rectangle(img0, [x,y], [x2,y2], (255,0,0), 2)
            display(Image.fromarray(cv2.cvtColor(img0,cv2.COLOR_BGR2RGB)))
    
    #answer
    image_id = os.path.basename(path).split('.jpg')[0]
    video_id, video_frame = image_id.split('-')
    img_path = f"input/train_images/video_{video_id}/{video_frame}.jpg"
    
    ans_img = cv2.imread(img_path)
    
    annos = df[df['image_id']==image_id]['annotations']
    annos = ast.literal_eval(annos.item())
    for anno in annos:
        x = anno['x']
        y = anno['y']
        w = anno['width']
        h = anno['height']
        x2 = x+w
        y2 = y+h
        cv2.rectangle(ans_img, [x,y], [x2,y2], (0,0,255), 2)
    display(Image.fromarray(cv2.cvtColor(ans_img,cv2.COLOR_BGR2RGB)))

In [ ]:
pred